In [1]:
from ctapipe.io import HDF5TableWriter
from ctapipe.core import Container, Field
import numpy as np

In [2]:
class VariousTypesContainer(Container):
    
    a_int = Field(int, 'int')
    a_float = Field(float, 'float')
    a_bool = Field(bool, 'bool')
    a_np_int = Field(np.int, 'np.int')
    a_np_float = Field(np.float, 'np.float')
    a_np_bool = Field(np.bool, 'np.bool')

In [3]:
def create_stream(n_event):
    
    data = VariousTypesContainer()
    for i in range(n_event):
        
        data.a_int = int(i)
        data.a_float = float(i)
        data.a_bool = (i % 2) == 0
        data.a_np_int = np.int(i)
        data.a_np_float = np.float(i)
        data.a_np_bool = np.bool((i % 2) == 0)
        
        yield data

In [4]:
for data in create_stream(2):
        
    for key, val in data.items():
        
        print('{}: {}, type : {}'.format(key, val, type(val)))

a_int: 0, type : <class 'int'>
a_np_bool: True, type : <class 'bool'>
a_float: 0.0, type : <class 'float'>
a_bool: True, type : <class 'bool'>
a_np_int: 0, type : <class 'int'>
a_np_float: 0.0, type : <class 'float'>
a_int: 1, type : <class 'int'>
a_np_bool: False, type : <class 'bool'>
a_float: 1.0, type : <class 'float'>
a_bool: False, type : <class 'bool'>
a_np_int: 1, type : <class 'int'>
a_np_float: 1.0, type : <class 'float'>


# Writing the Data

## This is not recommended

In [5]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)

h5_table.close()

In that case the file is not garenteed to close properly for instance if one does a mistake in the for loop. Let's just add a stupid mistake and see what happens.

In [6]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)
    0/0

h5_table.close()

ZeroDivisionError: division by zero

Now the file did not close properly. So let's try to correct the mistake and execute the code again.

In [7]:
h5_table = HDF5TableWriter('container.h5', group_name='data')

for data in create_stream(10):
    
    h5_table.write('table', data)

h5_table.close()

ValueError: The file 'container.h5' is already opened.  Please close it before reopening in write mode.

Ah it seems that the file did not close! Now I am stuck. Maybe I should restart the kernel? ahh no I don't want to loose everything. Can I just close it ?

In [8]:
h5_table.close()

It worked!

## Better to use context management

In [9]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
        
    for data in create_stream(10):
    
        h5_table.write('table', data)
print('Done')

Done


Now let me do the same mistake

In [10]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
    
    for data in create_stream(10):
        
        h5_table.write('table', data)
        0 / 0
print('Done')

ZeroDivisionError: division by zero

Let's fix it again...

In [11]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
        
    for data in create_stream(10):
    
        h5_table.write('table', data)
        
print('Done')

Done


Ok perfect my file closed automatically in this case!

In [12]:
!ls container.h5

container.h5


# Appending new Containers

To append some new containers we need to set the writing in append mode by using: 'mode=a'. But let's now first look at what happens if we don't.

In [13]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='w', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

container.h5 (File) ''
Last modif.: 'Tue Mar 20 16:24:10 2018'
Object Tree: 
/ (RootGroup) ''
/data_0 (Group) ''
/data_0/table (Table(0,)) 'Storage of VariousTypesContainer'

container.h5 (File) ''
Last modif.: 'Tue Mar 20 16:24:10 2018'
Object Tree: 
/ (RootGroup) ''
/data_1 (Group) ''
/data_1/table (Table(0,)) 'Storage of VariousTypesContainer'



In [14]:
!rm container.h5

Ok so the writer destroyed the content of the file each time it opens the file. Now let's try to append some data group to it! (using mode='a')

In [15]:
for i in range(2):

    with HDF5TableWriter('container.h5', mode='a', group_name='data_{}'.format(i)) as h5_table:
    
        for data in create_stream(10):
    
            h5_table.write('table', data)

        print(h5_table._h5file)

container.h5 (File) ''
Last modif.: 'Tue Mar 20 16:24:12 2018'
Object Tree: 
/ (RootGroup) ''
/data_0 (Group) ''
/data_0/table (Table(0,)) 'Storage of VariousTypesContainer'

container.h5 (File) ''
Last modif.: 'Tue Mar 20 16:24:12 2018'
Object Tree: 
/ (RootGroup) ''
/data_0 (Group) ''
/data_0/table (Table(10,)) 'Storage of VariousTypesContainer'
/data_1 (Group) ''
/data_1/table (Table(0,)) 'Storage of VariousTypesContainer'



So we can append some data groups. As long as the data group_name does not already exists. Let's try to overwrite the data group : data_1

In [16]:
with HDF5TableWriter('container.h5', mode='a', group_name='data_1') as h5_table:
    
    for data in create_stream(10):
    
        h5_table.write('table', data)

NodeError: group ``/`` already has a child node named ``data_1``

Good ! I cannot overwrite my data.

In [17]:
print(bool(h5_table._h5file.isopen))

False


# Reading the Data

In [18]:
from ctapipe.io import HDF5TableReader

def read(mode):
    
    print('reading mode {}'.format(mode))

    with HDF5TableReader('container.h5', mode=mode) as h5_table:

        for group_name in ['data_0/', 'data_1/']:

            group_name = '/{}table'.format(group_name)
            print(group_name)

            for data in h5_table.read(group_name, VariousTypesContainer()):

                print(data.as_dict())

reading mode r


OSError: File container.h5 cannot be opened in r mode

In [20]:
read('r')

reading mode r


OSError: File container.h5 cannot be opened in r mode

In [21]:
read('r+')

reading mode r+
/data_0/table
{'a_int': 0, 'a_bool': True, 'a_float': 0.0, 'a_np_bool': True, 'a_np_int': 0, 'a_np_float': 0.0}
{'a_int': 1, 'a_bool': False, 'a_float': 1.0, 'a_np_bool': False, 'a_np_int': 1, 'a_np_float': 1.0}
{'a_int': 2, 'a_bool': True, 'a_float': 2.0, 'a_np_bool': True, 'a_np_int': 2, 'a_np_float': 2.0}
{'a_int': 3, 'a_bool': False, 'a_float': 3.0, 'a_np_bool': False, 'a_np_int': 3, 'a_np_float': 3.0}
{'a_int': 4, 'a_bool': True, 'a_float': 4.0, 'a_np_bool': True, 'a_np_int': 4, 'a_np_float': 4.0}
{'a_int': 5, 'a_bool': False, 'a_float': 5.0, 'a_np_bool': False, 'a_np_int': 5, 'a_np_float': 5.0}
{'a_int': 6, 'a_bool': True, 'a_float': 6.0, 'a_np_bool': True, 'a_np_int': 6, 'a_np_float': 6.0}
{'a_int': 7, 'a_bool': False, 'a_float': 7.0, 'a_np_bool': False, 'a_np_int': 7, 'a_np_float': 7.0}
{'a_int': 8, 'a_bool': True, 'a_float': 8.0, 'a_np_bool': True, 'a_np_int': 8, 'a_np_float': 8.0}
{'a_int': 9, 'a_bool': False, 'a_float': 9.0, 'a_np_bool': False, 'a_np_int': 9,

In [22]:
read('a')

reading mode a
/data_0/table
{'a_int': 0, 'a_bool': True, 'a_float': 0.0, 'a_np_bool': True, 'a_np_int': 0, 'a_np_float': 0.0}
{'a_int': 1, 'a_bool': False, 'a_float': 1.0, 'a_np_bool': False, 'a_np_int': 1, 'a_np_float': 1.0}
{'a_int': 2, 'a_bool': True, 'a_float': 2.0, 'a_np_bool': True, 'a_np_int': 2, 'a_np_float': 2.0}
{'a_int': 3, 'a_bool': False, 'a_float': 3.0, 'a_np_bool': False, 'a_np_int': 3, 'a_np_float': 3.0}
{'a_int': 4, 'a_bool': True, 'a_float': 4.0, 'a_np_bool': True, 'a_np_int': 4, 'a_np_float': 4.0}
{'a_int': 5, 'a_bool': False, 'a_float': 5.0, 'a_np_bool': False, 'a_np_int': 5, 'a_np_float': 5.0}
{'a_int': 6, 'a_bool': True, 'a_float': 6.0, 'a_np_bool': True, 'a_np_int': 6, 'a_np_float': 6.0}
{'a_int': 7, 'a_bool': False, 'a_float': 7.0, 'a_np_bool': False, 'a_np_int': 7, 'a_np_float': 7.0}
{'a_int': 8, 'a_bool': True, 'a_float': 8.0, 'a_np_bool': True, 'a_np_int': 8, 'a_np_float': 8.0}
{'a_int': 9, 'a_bool': False, 'a_float': 9.0, 'a_np_bool': False, 'a_np_int': 9, 

OSError: File container.h5 cannot be opened in w mode

In [23]:
read('w')

reading mode w


OSError: File container.h5 cannot be opened in w mode